# 1. Apple Watch data cleaning

In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from collections import Counter
import math

In [2]:
apple = pd.read_csv('../../proyecto_final/data/apple_health_export/ActiveEnergyBurned.csv')

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_84298/1635027116.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  apple = pd.read_csv('../../proyecto_final/data/apple_health_export/ActiveEnergyBurned.csv')


In [3]:
apple = apple.astype({'startDate': 'datetime64[ns]','endDate': 'datetime64[ns]'})

In [4]:
apple['diff'] = apple['endDate'] - apple['startDate']

In [5]:
apple.drop(columns=['sourceName','sourceVersion','device','type', 'creationDate'], inplace= True)

In [6]:
apple.head(5)

,unit,startDate,endDate,value,diff
0,kcal,2016-05-28 08:00:00,2016-05-28 08:36:00,239.0,0 days 00:36:00
1,kcal,2020-11-05 13:40:00,2020-11-05 13:49:59,1.0,0 days 00:09:59
2,kcal,2020-11-05 15:30:00,2020-11-05 15:39:59,1.0,0 days 00:09:59
3,kcal,2020-11-05 15:40:00,2020-11-05 15:49:59,3.0,0 days 00:09:59
4,kcal,2020-11-05 15:50:00,2020-11-05 15:59:59,2.0,0 days 00:09:59


In [7]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336135 entries, 0 to 336134
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   unit       336135 non-null  object         
 1   startDate  336135 non-null  datetime64[ns] 
 2   endDate    336135 non-null  datetime64[ns] 
 3   value      336135 non-null  float64        
 4   diff       336135 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), object(1), timedelta64[ns](1)
memory usage: 12.8+ MB


In [8]:
apple['diff'][0].seconds/60

36.0

In [9]:
apple.value[0]/(apple['diff'][0].seconds/60)

6.638888888888889

In [10]:
apple['cal_min']=[apple.value[i]/(apple['diff'][i].seconds/60) for i in range(len(apple))]

apple.head()

,unit,startDate,endDate,value,diff,cal_min
0,kcal,2016-05-28 08:00:00,2016-05-28 08:36:00,239.0,0 days 00:36:00,6.638889
1,kcal,2020-11-05 13:40:00,2020-11-05 13:49:59,1.0,0 days 00:09:59,0.100167
2,kcal,2020-11-05 15:30:00,2020-11-05 15:39:59,1.0,0 days 00:09:59,0.100167
3,kcal,2020-11-05 15:40:00,2020-11-05 15:49:59,3.0,0 days 00:09:59,0.300501
4,kcal,2020-11-05 15:50:00,2020-11-05 15:59:59,2.0,0 days 00:09:59,0.200334


In [11]:
apple.drop(columns=['value','unit'], inplace= True)

In [12]:
apple.head()

,startDate,endDate,diff,cal_min
0,2016-05-28 08:00:00,2016-05-28 08:36:00,0 days 00:36:00,6.638889
1,2020-11-05 13:40:00,2020-11-05 13:49:59,0 days 00:09:59,0.100167
2,2020-11-05 15:30:00,2020-11-05 15:39:59,0 days 00:09:59,0.100167
3,2020-11-05 15:40:00,2020-11-05 15:49:59,0 days 00:09:59,0.300501
4,2020-11-05 15:50:00,2020-11-05 15:59:59,0 days 00:09:59,0.200334


In [13]:

l = [pd.date_range(start = apple.startDate[i], end=apple.endDate[i], freq="1min").strftime('%Y-%m-%d %H:%M') for i in tqdm(range(len(apple)))]

  0%|          | 0/336135 [00:00<?, ?it/s]

In [14]:
flat_list = [item for sublist in l for item in sublist]

In [15]:
df = pd.DataFrame()

In [16]:
df['date'] = flat_list

In [17]:
df.tail()

,date
538341,2022-11-11 08:20
538342,2022-11-11 08:21
538343,2022-11-11 08:22
538344,2022-11-11 08:23
538345,2022-11-11 08:24


In [18]:
apple.rename(columns={'startDate':'date'}, inplace=True)

In [19]:
apple.columns

Index(['date', 'endDate', 'diff', 'cal_min'], dtype='object')

In [20]:
apple.drop(columns=['endDate', 'diff'], inplace=True)

In [21]:
df = df.astype({'date': 'datetime64[ns]'})

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538346 entries, 0 to 538345
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    538346 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.1 MB


In [23]:
df = df.merge(apple,how='left', on= 'date')

In [24]:
df.cal_min.ffill(inplace=True)

# 2. Spotify data cleaning

In [25]:
mus = pd.read_json('../../proyecto_final/data/mydataspotify/StreamingHistory0.json')

In [26]:
mus.shape

(10000, 4)

In [27]:
music2 = pd.read_json('../../proyecto_final/data/mydataspotify/StreamingHistory1.json')

In [28]:
music2.shape

(6530, 4)

In [29]:
music = pd.concat([mus, music2], axis=0, ignore_index=True)

In [30]:
music.shape

(16530, 4)

In [31]:
music.rename(columns={'endTime':'date'}, inplace=True)

In [32]:
music = music.astype({'date': 'datetime64[ns]'})

In [33]:
music.shape

(16530, 4)

In [34]:
df.head()

,date,cal_min
0,2016-05-28 08:00:00,6.638889
1,2016-05-28 08:01:00,6.638889
2,2016-05-28 08:02:00,6.638889
3,2016-05-28 08:03:00,6.638889
4,2016-05-28 08:04:00,6.638889


In [35]:
df.shape

(538364, 2)

In [36]:
music.shape

(16530, 4)

In [37]:
music.loc[music['date']=='2022-08-07 11:54:00']

,date,artistName,trackName,msPlayed
11162,2022-08-07 11:54:00,Boza,Ella - Remix,330478
11163,2022-08-07 11:54:00,Joje,1234,120000
11164,2022-08-07 11:54:00,TM,Nextbike,165340
11165,2022-08-07 11:54:00,KASIMIR1441,TEK TEK,109924
11166,2022-08-07 11:54:00,Jhayco,Dile (Homenaje),228933
11167,2022-08-07 11:54:00,Jhayco,Tokyo,11912
11168,2022-08-07 11:54:00,Rauw Alejandro,Desesperados,224997
11169,2022-08-07 11:54:00,Rauw Alejandro,Desesperados,302
11170,2022-08-07 11:54:00,Omar Montes,Carita Morena,111300
11171,2022-08-07 11:54:00,Robinio,Cash,145968


In [38]:
music.drop_duplicates(subset='date', inplace=True)

In [39]:
df.value_counts()

date                 cal_min 
2022-09-25 15:11:00  4.200000    24
2022-07-15 14:38:00  5.140000    24
2022-09-24 11:38:00  3.420000    24
2022-07-15 14:34:00  2.860000    24
2022-07-15 14:32:00  7.830000    24
                                 ..
2021-12-14 11:39:00  0.060500     1
2021-12-14 11:40:00  0.060500     1
2021-12-14 11:41:00  0.060500     1
2021-12-14 11:42:00  0.060500     1
2022-11-11 08:24:00  0.421935     1
Length: 318286, dtype: int64

In [40]:
df.loc[df['date']=='2022-09-25 15:10:00']

,date,cal_min
457646,2022-09-25 15:10:00,2.3
457647,2022-09-25 15:10:00,2.3
457648,2022-09-25 15:10:00,2.3
457649,2022-09-25 15:10:00,2.3
457650,2022-09-25 15:10:00,2.3
457651,2022-09-25 15:10:00,2.3
457652,2022-09-25 15:10:00,2.3
457653,2022-09-25 15:10:00,2.3
457654,2022-09-25 15:10:00,2.3
457655,2022-09-25 15:10:00,2.3


In [41]:
df.drop_duplicates(subset='date', inplace=True)

In [42]:
df.date.value_counts().head()

2016-05-28 08:00:00    1
2022-06-13 10:21:00    1
2022-06-13 10:28:00    1
2022-06-13 10:27:00    1
2022-06-13 10:26:00    1
Name: date, dtype: int64

In [43]:
music.shape

(15028, 4)

In [44]:
music['sgPlayed'] = music['msPlayed']/1000

In [45]:
music['minPlayed'] = music['sgPlayed']/60

# Music to burn calories Analysis

In [46]:
music = music.merge(df ,how='left', on= 'date')

In [47]:
nan_cols=music.isna().sum()

nan_cols[nan_cols>0]

cal_min    4374
dtype: int64

In [48]:
music.cal_min.ffill(inplace=True)

In [49]:
nan_cols=music.isna().sum()

nan_cols[nan_cols>0]

Series([], dtype: int64)

In [50]:
music.to_csv('../../proyecto_final/data/clean_data/usermusic&usercal.csv', index=False)

In [51]:
music.shape

(15028, 7)

In [52]:
music.head()

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min
0,2021-11-12 15:22:00,Myke Towers,Experimento,106394,106.394,1.773233,0.316418
1,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588
2,2021-11-13 08:56:00,Tame Impala,The Less I Know The Better,213141,213.141,3.552350,0.210588
3,2021-11-13 09:33:00,Goth Babe,Canary Islands,3904,3.904,0.065067,0.316364
4,2021-11-13 09:36:00,Goth Babe,Canary Islands,201042,201.042,3.350700,0.316364


## Most calories burned while listening a song

In [53]:
music.loc[music['cal_min'] == music['cal_min'].max()]

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min
13872,2022-10-22 15:26:00,Florence + The Machine,You've Got The Love,168666,168.666,2.8111,9.93


Florence and the machine - You've got the love es la canción que he escuchado en el momento en el que estaba quemando una mayor cantidad de calorías.

In [54]:
music.sort_values(by=['cal_min'], ascending=False)[:10]

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min
13872,2022-10-22 15:26:00,Florence + The Machine,You've Got The Love,168666,168.666,2.811100,9.93
13102,2022-10-03 14:44:00,Jamule,KISSENSCHLACHT,171069,171.069,2.851150,9.81
12822,2022-09-25 09:12:00,Rayo & Toby,Ninfómana,9930,9.930,0.165500,8.88
13105,2022-10-03 14:51:00,Miksu / Macloud,Nachts wach (Lila Wolken Bootleg),158080,158.080,2.634667,8.79
13454,2022-10-13 12:33:00,Sam Smith,Unholy (feat. Kim Petras),156943,156.943,2.615717,8.64
13451,2022-10-13 12:24:00,Beyoncé,Daddy Lessons,287986,287.986,4.799767,8.55
14075,2022-10-24 17:59:00,Beyoncé,CUFF IT,225388,225.388,3.756467,8.52
13452,2022-10-13 12:26:00,Beyoncé,Yoncé - Homecoming Live,68351,68.351,1.139183,8.40
13453,2022-10-13 12:31:00,Beyoncé,Daddy Lessons,287986,287.986,4.799767,8.40
13321,2022-10-10 12:29:00,Ben Böhmer,In Memoriam - Live from Printworks London,377499,377.499,6.291650,8.22


In [55]:
music.shape

(15028, 7)

## Most streamed artists and calories burned

In [56]:
artist = music.groupby(['artistName'])['cal_min'].sum()

In [57]:
artist = pd.DataFrame(artist)

In [58]:
artist.reset_index(drop=False, inplace=True)

In [59]:
artist.sort_values(by=['cal_min'], ascending=False)[:10]

,artistName,cal_min
219,Bad Bunny,395.875314
1866,ROSALÍA,350.443970
1854,Quevedo,258.186482
274,Beyoncé,177.302282
2368,WILLOW,148.251626
1877,Rauw Alejandro,135.244929
190,Avril Lavigne,117.463126
1484,Manuel Turizo,111.631728
92,Alicia Keys,106.578428
300,Bizarrap,103.983952


Esta lista muestra los artistas con los que he quemado más calorías.

Pero... ¿serán estos los artistas con más escuchas?

In [60]:
moststreamed = music.groupby(['artistName'])['minPlayed'].sum()

In [61]:
moststreamed = pd.DataFrame(moststreamed)

In [62]:
moststreamed.reset_index(drop=False, inplace=True)

In [63]:
moststreamed.sort_values(by=['minPlayed'], ascending=False)[:10]

,artistName,minPlayed
219,Bad Bunny,1429.738850
2352,Viene y Va con Dani G Schulz,1057.849450
1866,ROSALÍA,1056.325467
1854,Quevedo,766.343367
1561,Mi Mejor Versión con Isa Garcia,616.482517
92,Alicia Keys,594.901383
775,Estas Rica,556.932400
1877,Rauw Alejandro,552.405783
827,Fitness Stuff (for normal people),478.182717
1073,Jazz Relajante,471.831100


In [64]:
mostS = moststreamed.sort_values(by=['minPlayed'], ascending=False)[:10]

In [65]:
mostS = list(mostS.artistName)

In [66]:
mostC = artist.sort_values(by=['cal_min'], ascending=False)[:10]

In [67]:
mostC = list(mostC.artistName)

In [68]:
counterA = Counter(mostS)
counterB = Counter(mostC)

In [69]:
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [70]:
counter_cosine_similarity(counterA, counterB)

0.4999999999999999

Como se puede observar, hay menos del 50% de similaridad entre la lista de artistas más escuchados por el usuario y aquellos artistas con los que quema más calorías, por lo que podemos mantener la hipótesis de que estos artistas prefiere escucharlos cuando entrena o realiza actividad física.

## Most streamed tracks and calories burned

In [71]:
track = music.groupby(['trackName'])['cal_min'].sum()

In [72]:
track = pd.DataFrame(track)

In [73]:
track.reset_index(drop=False, inplace=True)

In [74]:
track.sort_values(by=['cal_min'], ascending=False)[:10]

,trackName,cal_min
2531,La Bachata,94.367196
3732,Piel de Cordero,59.640876
5182,Wait a Minute!,51.061894
5593,hover like a GODDESS,50.335614
879,"Cayó La Noche (feat. Cruz Cafuné, Abhir Hathi,...",48.057017
3481,Ojitos Lindos,47.935293
1145,Daddy Lessons,45.052125
1129,DESPECHÁ,43.410375
2241,I'm Good (Blue),40.961847
2508,LA COMBI VERSACE,40.904029


Esta lista muestra las canciones con los que he quemado más calorías. Pero... ¿serán estas las canciones con más escuchas?

In [75]:
trackmoststreamed = music.groupby(['trackName'])['minPlayed'].sum()

In [76]:
trackmoststreamed = pd.DataFrame(trackmoststreamed)

In [77]:
trackmoststreamed.reset_index(drop=False, inplace=True)

In [78]:
trackmoststreamed.sort_values(by=['minPlayed'], ascending=False)[:10]

,trackName,minPlayed
879,"Cayó La Noche (feat. Cruz Cafuné, Abhir Hathi,...",398.207183
3732,Piel de Cordero,223.503667
3481,Ojitos Lindos,208.195033
8,#55 Cómo cambiar hábitos con gracia,166.790783
3411,Nostálgico,152.148300
1534,"Ep 24 / How Much Protein You Need to Lose Fat,...",147.602967
2531,La Bachata,136.581300
5151,Volando - Remix,119.104567
3066,Mi Luz (ft. Rels B),118.128300
1776,Formentera,109.635967


In [79]:
mostS = trackmoststreamed.sort_values(by=['minPlayed'], ascending=False)[:10]

In [80]:
mostS = list(mostS.trackName)

In [81]:
mostC = track.sort_values(by=['cal_min'], ascending=False)[:10]

In [82]:
mostC = list(mostC.trackName)

In [83]:
counterC = Counter(mostS)
counterD = Counter(mostC)

In [84]:
counter_cosine_similarity(counterC, counterD)

0.3999999999999999

Como se puede observar, hay menos del 20% de similaridad entre la lista de canciones más escuchados por el usuario y aquellas canciones con las que quema más calorías, por lo que podemos mantener la hipótesis de que estas canciones prefiere escucharlas cuando entrena o realiza actividad física.

## Próximos pasos

- testeo de hipótesis
- información de kaggle (genero + danceability + energy + loudness + speechiness)
- scrappeo shazam (género)
- API spotipy (¿puedo sacar ccas de la cancion y género?) - vamos a hacer esto

Más datos:
- Información de heart rate
- Información de entrenamientos

In [76]:
heart_ori = pd.read_csv('../../proyecto_final/data/apple_health_export/HeartRate.csv')

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_5011/826925066.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  heart_ori = pd.read_csv('../../proyecto_final/data/apple_health_export/HeartRate.csv')


In [77]:
heart = heart_ori.copy()

In [78]:
heart.head()

,sourceName,sourceVersion,device,type,unit,creationDate,startDate,endDate,value
0,Zepp Life,202010291402.0,NaN,HeartRate,count/min,2020-11-05 19:18:44 +0100,2020-11-03 00:00:00 +0100,2020-11-03 00:00:59 +0100,73.0
1,Zepp Life,202010291402.0,NaN,HeartRate,count/min,2020-11-05 19:18:44 +0100,2020-11-03 00:01:00 +0100,2020-11-03 00:01:59 +0100,69.0
2,Zepp Life,202010291402.0,NaN,HeartRate,count/min,2020-11-05 19:18:44 +0100,2020-11-03 00:02:00 +0100,2020-11-03 00:02:59 +0100,69.0
3,Zepp Life,202010291402.0,NaN,HeartRate,count/min,2020-11-05 19:18:44 +0100,2020-11-03 00:03:00 +0100,2020-11-03 00:03:59 +0100,70.0
4,Zepp Life,202010291402.0,NaN,HeartRate,count/min,2020-11-05 19:18:44 +0100,2020-11-03 00:04:00 +0100,2020-11-03 00:04:59 +0100,70.0


In [79]:
heart.rename(columns={'value': 'count_min'}, inplace=True)

In [80]:
heart = heart.drop(columns=['sourceName','sourceVersion','device','type','unit','creationDate'])

In [81]:
heart.tail()

,startDate,endDate,count_min
542423,2022-11-11 09:19:33 +0100,2022-11-11 09:19:33 +0100,116.0
542424,2022-11-11 09:20:10 +0100,2022-11-11 09:20:10 +0100,102.0
542425,2022-11-11 09:20:17 +0100,2022-11-11 09:20:17 +0100,110.0
542426,2022-11-11 09:20:18 +0100,2022-11-11 09:20:18 +0100,108.0
542427,2022-11-11 09:20:27 +0100,2022-11-11 09:20:27 +0100,96.0


In [82]:
heart = heart.astype({'startDate': 'datetime64[ns]', 'endDate':'datetime64[ns]'})

In [83]:
heart['diff'] = heart['endDate']-heart['startDate']

In [84]:
heart.sort_values(['diff'], ascending=False)[:5]

,startDate,endDate,count_min,diff
463982,2022-01-06 10:03:27,2022-01-06 10:41:19,121.4450,0 days 00:37:52
473596,2022-02-21 16:49:06,2022-02-21 17:16:37,109.0740,0 days 00:27:31
463118,2022-01-04 13:31:39,2022-01-04 13:57:36,115.1870,0 days 00:25:57
463114,2022-01-04 12:20:10,2022-01-04 12:45:24,116.8250,0 days 00:25:14
497955,2022-04-25 18:45:51,2022-04-25 19:10:21,97.5628,0 days 00:24:30


In [85]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542428 entries, 0 to 542427
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   startDate  542428 non-null  datetime64[ns] 
 1   endDate    542428 non-null  datetime64[ns] 
 2   count_min  542428 non-null  float64        
 3   diff       542428 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), timedelta64[ns](1)
memory usage: 16.6 MB


In [86]:
heart.shape

(542428, 4)

In [153]:
heart.tail()

,count_min
date,
2022-11-11 08:19:33,116.0
2022-11-11 08:20:10,102.0
2022-11-11 08:20:17,110.0
2022-11-11 08:20:18,108.0
2022-11-11 08:20:27,96.0


In [155]:
l = pd.date_range(start='2020-11-02 23:00:00', end='2022-11-11 08:20:00', freq='1min').strftime('%Y-%m-%d %H:%M')

In [157]:
l = pd.DataFrame(l)

In [165]:
l.rename(columns={0:'date'}, inplace=True)

In [168]:
l= l.astype({'date':'datetime64[ns]'})

In [170]:
l = l.merge(heart, on='date', how='left')

In [173]:
l.count_min.ffill(inplace=True)

In [174]:
nan_cols=l.isna().sum()
nan_cols[nan_cols>0]

Series([], dtype: int64)

In [175]:
data = pd.read_csv('../../proyecto_final/data/clean_data/usermusic&usercal.csv')

In [176]:
data.shape

(15028, 7)

In [177]:
data.head()

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min
0,2021-11-12 15:22:00,Myke Towers,Experimento,106394,106.394,1.773233,0.316418
1,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588
2,2021-11-13 08:56:00,Tame Impala,The Less I Know The Better,213141,213.141,3.552350,0.210588
3,2021-11-13 09:33:00,Goth Babe,Canary Islands,3904,3.904,0.065067,0.316364
4,2021-11-13 09:36:00,Goth Babe,Canary Islands,201042,201.042,3.350700,0.316364


In [178]:
data = data.astype({'date':'datetime64[ns]'})

In [179]:
data = data.merge(l, on='date', how='left')

In [180]:
data.head()

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min,count_min
0,2021-11-12 15:22:00,Myke Towers,Experimento,106394,106.394,1.773233,0.316418,100.0
1,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588,67.0
2,2021-11-13 08:56:00,Tame Impala,The Less I Know The Better,213141,213.141,3.552350,0.210588,67.0
3,2021-11-13 09:33:00,Goth Babe,Canary Islands,3904,3.904,0.065067,0.316364,67.0
4,2021-11-13 09:36:00,Goth Babe,Canary Islands,201042,201.042,3.350700,0.316364,67.0


In [182]:
data.count_min.ffill(inplace=True)

In [183]:
nan_cols= data.isna().sum()
nan_cols[nan_cols>0]

Series([], dtype: int64)

# Análisis de la data

In [188]:
data.sort_values(by='count_min', ascending=False)[:10]

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min,count_min
13320,2022-10-10 12:23:00,Mark Forster,Mellow Mellow,15130,15.130,0.252167,3.42,177.0
13318,2022-10-10 12:16:00,Munic HB,Pico y Pala,82407,82.407,1.373450,2.86,175.0
13319,2022-10-10 12:18:00,Feid,Feliz Cumpleaños Ferxxo,113450,113.450,1.890833,2.86,175.0
13454,2022-10-13 12:33:00,Sam Smith,Unholy (feat. Kim Petras),156943,156.943,2.615717,8.64,171.0
13455,2022-10-13 12:37:00,Mega,Chariot,216755,216.755,3.612583,6.80,171.0
13317,2022-10-10 12:14:00,DELLAFUENTE,K alegría,140575,140.575,2.342917,2.86,170.0
13120,2022-10-03 15:33:00,Luar La L,Caile,141340,141.340,2.355667,5.14,168.0
9885,2022-07-31 11:44:00,Megan Thee Stallion,Sweetest Pie,201334,201.334,3.355567,5.78,167.0
9886,2022-07-31 11:47:00,Cat Burns,go (feat. Sam Smith),215841,215.841,3.597350,5.78,167.0
12699,2022-09-24 11:01:00,Jessie J,Bang Bang,199386,199.386,3.323100,4.52,164.0


In [206]:
heartar = data.groupby(['trackName'])['count_min'].median()

In [207]:
heartar = pd.DataFrame(heartar)

In [208]:
heartar.sort_values(by='count_min', ascending=True)[:10]

,count_min
trackName,
Don't Leave Me Lonely (feat. YEBBA),59.0
Being Human - Single Version,59.0
Just Another Thing We Don't Talk About,59.0
Dilemma (feat. Kelly Rowland),59.0
Poison,59.0
Hot In Herre,59.0
Dilemma,59.0
No Scrubs,59.0
Sunrise___,59.0
